<a href="https://colab.research.google.com/github/utkarsh-mishra19/pytorch/blob/main/pytorch_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

Skipping, found downloaded files in "./rice-type-classification" (use force=True to force download)


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader # Corrected import
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
device = "cuda" if torch.cuda.is_available else "cpu"
print(device)

cuda


In [ ]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [ ]:
data_df.dropna(inplace= True)
data_df.drop(['id'],axis =1,inplace = True)
print(data_df.shape)

(18185, 11)


In [ ]:
data_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [ ]:
print(data_df["Class"].unique())

[1 0]


normilaziation:deviding each value in column by largest value in that colmn

In [ ]:
original_df = data_df.copy()
for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()
data_df.head()


,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [ ]:
x = np.array(data_df.iloc[: ,: -1])
y = np.array(data_df.iloc[:, -1])

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3)
x_test,x_val,y_test,y_val = train_test_split(x_test,y_test,test_size = 0.5)

In [ ]:
class dataset(Dataset):
  def __init__(self,x,y) -> None:
    self.x = torch.tensor(x,dtype=torch.float32).to(device)
    self.y = torch.tensor(y,dtype=torch.float32).to(device)
  def __len__(self):
    return len(self.x)
  def __getitem__(self, index):
    return self.x[index],self.y[index]

In [ ]:
training_data = dataset(x_train,y_train)
validation_data=dataset(x_val,y_val)
testing_data=dataset(x_test,y_test)


In [ ]:
training_dataloader = DataLoader(training_data,batch_size=32,shuffle=True) # Corrected usage
validation_dataloader = DataLoader(validation_data,batch_size=32,shuffle=True) # Corrected usage
testing_dataloader = DataLoader(testing_data,batch_size=32,shuffle=True) # Corrected usage


In [ ]:
class mymodel(nn.Module):
  def __init__(self):
    super(mymodel,self).__init__()
    self.input_layer = nn.Linear(x.shape[1],15)
    self.linear = nn.Linear(15,1)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x):
     x = self.input_layer(x)
     x = self.linear(x)
     x = self.sigmoid(x)
     return x
model = mymodel().to(device)

In [ ]:
criterion = nn.BCELoss()
optimizer= Adam(model.parameters(),lr=0.001)



In [ ]:

epochs = 10
for epoch in range(epochs):
  total_loss_train = 0
  total_loss_val = 0
  total_acc_train = 0
  total_acc_val = 0
  for data in training_dataloader:
    inputs,labels = data
    prediction = model(inputs).squeeze(1)
    batchloss = criterion(prediction,labels)
    total_loss_train += batchloss.item()
    acc = ((prediction).round() == labels).sum().item() # Corrected accuracy calculation
    total_acc_train += acc
    batchloss.backward()
    optimizer.step()
    optimizer.zero_grad()
  with torch.no_grad():
    for data in validation_dataloader:
      inputs,labels = data
      prediction = model(inputs).squeeze(1) # Corrected typo
      batchloss = criterion(prediction,labels)
      total_loss_val += batchloss.item() # Corrected typo
      acc = ((prediction).round() == labels).sum().item() # Corrected accuracy calculation
      total_acc_val += acc

  print(f'''Epoch no.{epoch+1} Train loss.{round(total_loss_train/100,4)} Train accuracy.{round(total_acc_train/training_data.__len__()*100,4)}
        Validation loss.{round(total_loss_val/100,4)} Validation accuracy.{round(total_acc_val/validation_data.__len__()*100,4)}''')
  print("-"*50)


Epoch no.1 Train loss.0.1647 Train accuracy.98.5938
        Validation loss.0.0375 Validation accuracy.98.3871
--------------------------------------------------
Epoch no.2 Train loss.0.1646 Train accuracy.98.5938
        Validation loss.0.038 Validation accuracy.98.4238
--------------------------------------------------
Epoch no.3 Train loss.0.165 Train accuracy.98.5545
        Validation loss.0.0372 Validation accuracy.98.5337
--------------------------------------------------
Epoch no.4 Train loss.0.1639 Train accuracy.98.6566
        Validation loss.0.0376 Validation accuracy.98.5337
--------------------------------------------------
Epoch no.5 Train loss.0.165 Train accuracy.98.5938
        Validation loss.0.0372 Validation accuracy.98.4971
--------------------------------------------------
Epoch no.6 Train loss.0.165 Train accuracy.98.5623
        Validation loss.0.0377 Validation accuracy.98.5337
--------------------------------------------------
Epoch no.7 Train loss.0.1656 Tra

In [ ]:
with torch.no_grad():
  total_loss_test = 0
  total_acc_test = 0
  for data in testing_dataloader:
    inputs,labels = data
    prediction = model(inputs).squeeze(1)
    batchloss = criterion(prediction,labels)
    total_loss_test += batchloss.item()
    acc = ((prediction).round() == labels).sum().item() # Corrected accuracy calculation
    total_acc_test += acc
  print(f'''Test loss.{round(total_loss_test/len(testing_dataloader),4)} Test accuracy.{round(total_acc_test/testing_data.__len__()*100,4)}''') # Averaging the loss
  print("-"*50)

Test loss.0.0345 Test accuracy.98.937
--------------------------------------------------


In [ ]:
ooriginal_df = original_df.copy()
for column in ooriginal_df.columns:
  ooriginal_df[column]=ooriginal_df[column]/ooriginal_df[column].abs().max()
x = np.array(ooriginal_df.iloc[: ,: -1])
y = np.array(ooriginal_df.iloc[:, -1])
x_test, _, y_test, _ = train_test_split(x, y, test_size=1, random_state=42) # Get the entire dataset as test set
testing_data=dataset(x_test,y_test)
testing_dataloader = DataLoader(testing_data,batch_size=32,shuffle=True) # Corrected usage
with torch.no_grad():
  test_loss = 0
  test_acc = 0
  for data in testing_dataloader:
    inputs,labels = data
    prediction = model(inputs).squeeze(1)
    batchloss = criterion(prediction,labels)
    acc = (prediction.round() == labels).sum().item() # Corrected accuracy calculation
    test_loss += batchloss.item()
    test_acc += acc
  print(f'''Test loss.{round(test_loss/len(testing_dataloader),4)} Test accuracy.{round(test_acc/testing_data.__len__()*100,4)}''') # Averaging the loss
  print("-"*50)

Test loss.0.04 Test accuracy.98.6307
--------------------------------------------------


In [ ]:
ooriginal_df.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,0.444368,0.503404,0.775435,0.744658,0.424873,0.666610,0.741661,0.537029,0.844997,0.368316,1.0
1,0.281293,0.407681,0.622653,0.750489,0.273892,0.530370,0.804230,0.409661,0.919215,0.371471,1.0
2,0.298531,0.416421,0.630442,0.756341,0.284520,0.546380,0.856278,0.412994,0.959862,0.374747,1.0
3,0.300979,0.420463,0.629049,0.764024,0.286791,0.548616,0.883772,0.414262,0.961818,0.379222,1.0
4,0.361704,0.464626,0.682901,0.775033,0.345385,0.601418,0.867808,0.452954,0.966836,0.386007,1.0


In [ ]:
pred = model(torch.tensor([0.3,	0.4,	0.6,	0.76,	0.2,	0.54,	0.88,	0.4,	0.9	,0.372],dtype = torch.float32).to(device))
pred.round().item()

1.0